# Import

In [1]:
%matplotlib inline
from scipy.stats import norm
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
sns.set()

# Data Retrieval

In [2]:
#load the csv data in a dataframe
DATA_PATH=".//dataset//TCGA"
DATA_NAME="clinical_data_coad.tsv"

import pandas as pd

def load_raw_data(data_path, data_name):
    csv_path = os.path.join(data_path, data_name)
    return pd.read_csv(csv_path,sep="\t")

df=load_raw_data(DATA_PATH,DATA_NAME)

# Data Info

In [3]:
df.head()

,Study ID,Patient ID,Sample ID,Diagnosis Age,Neoplasm Disease Stage American Joint Committee on Cancer Code,American Joint Committee on Cancer Publication Version Type,Aneuploidy Score,Buffa Hypoxia Score,Cancer Type,TCGA PanCanAtlas Cancer Type Acronym,...,9p Status,9q Status,Subtype,Tissue Prospective Collection Indicator,Tissue Retrospective Collection Indicator,Tissue Source Site,Tumor Disease Anatomic Site,Tumor Type,Patient Weight,Winter Hypoxia Score
0,coadread_tcga_pan_can_atlas_2018,TCGA-3L-AA1B,TCGA-3L-AA1B-01,61.0,STAGE I,7TH,19.0,5.0,Colorectal Adenocarcinoma,COAD,...,Not Called,Not Called,COAD_CIN,Yes,No,3L,Colon,Colon Adenocarcinoma,63.300,-12.0
1,coadread_tcga_pan_can_atlas_2018,TCGA-4N-A93T,TCGA-4N-A93T-01,67.0,STAGE IIIB,7TH,13.0,-15.0,Colorectal Adenocarcinoma,COAD,...,Not Called,Not Called,COAD_CIN,Yes,No,4N,Colon,Colon Adenocarcinoma,134.000,-6.0
2,coadread_tcga_pan_can_atlas_2018,TCGA-4T-AA8H,TCGA-4T-AA8H-01,42.0,STAGE IIA,7TH,18.0,15.0,Colorectal Adenocarcinoma,COAD,...,Not Called,Not Called,COAD_CIN,No,Yes,4T,Colon,"Colon Adenocarcinoma, Mucinous Type",107.956,-4.0
3,coadread_tcga_pan_can_atlas_2018,TCGA-5M-AAT4,TCGA-5M-AAT4-01,74.0,STAGE IV,6TH,18.0,35.0,Colorectal Adenocarcinoma,COAD,...,Gained,Not Called,COAD_CIN,No,Yes,5M,Colon,Colon Adenocarcinoma,NaN,32.0
4,coadread_tcga_pan_can_atlas_2018,TCGA-5M-AAT5,TCGA-5M-AAT5-01,NaN,NaN,NaN,10.0,33.0,Colorectal Adenocarcinoma,COAD,...,Not Called,Not Called,COAD_CIN,NaN,NaN,5M,NaN,Colon Adenocarcinoma,NaN,18.0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 594 entries, 0 to 593
Data columns (total 99 columns):
 #   Column                                                                                       Non-Null Count  Dtype  
---  ------                                                                                       --------------  -----  
 0   Study ID                                                                                     594 non-null    object 
 1   Patient ID                                                                                   594 non-null    object 
 2   Sample ID                                                                                    594 non-null    object 
 3   Diagnosis Age                                                                                592 non-null    float64
 4   Neoplasm Disease Stage American Joint Committee on Cancer Code                               580 non-null    object 
 5   American Joint Committee on Cancer P

In [5]:
df.pivot_table(index=['Overall Survival Status'],columns=['Patient ID'],values='Overall Survival (Months)')

Patient ID,TCGA-3L-AA1B,TCGA-4N-A93T,TCGA-4T-AA8H,TCGA-5M-AAT4,TCGA-5M-AAT6,TCGA-5M-AATE,TCGA-A6-2671,TCGA-A6-2672,TCGA-A6-2674,TCGA-A6-2675,...,TCGA-QG-A5YV,TCGA-QG-A5YW,TCGA-QG-A5YX,TCGA-QG-A5Z1,TCGA-QG-A5Z2,TCGA-QL-A97D,TCGA-RU-A8FL,TCGA-SS-A7HO,TCGA-T9-A92H,TCGA-WS-AB45
Overall Survival Status,,,,,,,,,,,,,,,,,,,,,
0:LIVING,15.616267,4.799947,12.657396,NaN,NaN,39.451622,NaN,46.651544,43.758425,43.429661,...,42.772134,29.457211,32.974981,NaN,31.298287,21.89565,38.695466,60.130848,11.901239,70.02663
1:DECEASED,NaN,NaN,NaN,1.610941,9.534142,NaN,43.758425,NaN,NaN,NaN,...,NaN,NaN,NaN,8.416346,NaN,NaN,NaN,NaN,NaN,NaN


# Feature Select Numeric and Categorical

In [6]:
featDrop=[0,1,2,5,6,12,14,22,23,25,26,27,36]
df_sel=df.drop(df.columns[featDrop],axis=1)

colNum=[]
colCat=[]

#separeta cat and num features
from pandas.api.types import is_numeric_dtype
for colname in df_sel.columns:
    if is_numeric_dtype(df_sel[colname]):
        colNum.append(colname)
    else:
        colCat.append(colname)


#sel Num feat
df_sel_num=df_sel[colNum].copy()

#sel cat feat
df_sel_cat=df_sel[colCat].copy()

In [7]:
df_sel_num.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 594 entries, 0 to 593
Data columns (total 15 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   Diagnosis Age                                 592 non-null    float64
 1   Buffa Hypoxia Score                           365 non-null    float64
 2   Birth from Initial Pathologic Diagnosis Date  590 non-null    float64
 3   Disease Free (Months)                         223 non-null    float64
 4   Months of disease-specific survival           590 non-null    float64
 5   Fraction Genome Altered                       583 non-null    float64
 6   MSI MANTIS Score                              557 non-null    float64
 7   MSIsensor Score                               584 non-null    float64
 8   Mutation Count                                528 non-null    float64
 9   Overall Survival (Months)                     590 non-null    flo

# Data Cleaning 2 (Numerical: Missing Values and Features Scaling) 

In [8]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler,MinMaxScaler 


#obj imp
imputer = SimpleImputer(strategy='median')

#obj sca
scaler=MinMaxScaler()


#fit imputer
imputer.fit(df_sel_num)
#trans
arr_sel_num_imp=imputer.transform(df_sel_num)
#fit scaling
scaler.fit(arr_sel_num_imp)
feat_sel_num_imp=scaler.transform(arr_sel_num_imp)
df_sel_num_imp=pd.DataFrame(feat_sel_num_imp,columns=df_sel_num.columns)

# Data Cleaning 3 (Missing Value for Categorical Features and OneHotEncoder)

In [9]:
#Fill missing data with None (we don't add extra bias)
df_sel_cat.fillna('None',inplace=True)
df_sel_cat.info()

#remove Overall Survival Status (for classification)
df_status=df_sel_cat['Overall Survival Status'].copy()
df_sel_cat_imp=df_sel_cat.drop('Overall Survival Status',axis=1)


#OneHotEncoder
from sklearn.preprocessing import OneHotEncoder

#from cat to ohe
encoderOHE=OneHotEncoder(sparse=False)
encoderOHE.fit(df_sel_cat_imp)
feat_sel_cat=encoderOHE.transform(df_sel_cat_imp)



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 594 entries, 0 to 593
Data columns (total 71 columns):
 #   Column                                                                     Non-Null Count  Dtype 
---  ------                                                                     --------------  ----- 
 0   Neoplasm Disease Stage American Joint Committee on Cancer Code             594 non-null    object
 1   Cancer Type                                                                594 non-null    object
 2   TCGA PanCanAtlas Cancer Type Acronym                                       594 non-null    object
 3   Cancer Type Detailed                                                       594 non-null    object
 4   Center of sequencing                                                       594 non-null    object
 5   Disease Free Status                                                        594 non-null    object
 6   Disease-specific Survival status                                  

# Train, Val, Test datasets

In [21]:
from sklearn.model_selection import train_test_split
import numpy as np
X=np.concatenate((feat_sel_num_imp, feat_sel_cat), axis=1)
y=df_status

X_train, X_test, y_train, y_test = train_test_split( X, y , test_size = 0.2 , random_state = 41)

# Score Func

In [ ]:
def display_scores(scores):
    print('Scores:', scores)
    print('Mean:', scores.mean())
    print('Standard deviation:', scores.std())

# Classifier ML Models

# Learning Curve

In [ ]:

def plot_learning_curve(estimator, title, X, y, axes=None, ylim=None, cv=None,
                        n_jobs=None, train_sizes=np.linspace(.1, 1.0, 5)):
    """
    Generate 3 plots: the test and training learning curve, the training
    samples vs fit times curve, the fit times vs score curve.

    Parameters
    ----------
    estimator : object type that implements the "fit" and "predict" methods
        An object of that type which is cloned for each validation.

    title : string
        Title for the chart.

    X : array-like, shape (n_samples, n_features)
        Training vector, where n_samples is the number of samples and
        n_features is the number of features.

    y : array-like, shape (n_samples) or (n_samples, n_features), optional
        Target relative to X for classification or regression;
        None for unsupervised learning.

    axes : array of 3 axes, optional (default=None)
        Axes to use for plotting the curves.

    ylim : tuple, shape (ymin, ymax), optional
        Defines minimum and maximum yvalues plotted.

    cv : int, cross-validation generator or an iterable, optional
        Determines the cross-validation splitting strategy.
        Possible inputs for cv are:

          - None, to use the default 5-fold cross-validation,
          - integer, to specify the number of folds.
          - :term:`CV splitter`,
          - An iterable yielding (train, test) splits as arrays of indices.

        For integer/None inputs, if ``y`` is binary or multiclass,
        :class:`StratifiedKFold` used. If the estimator is not a classifier
        or if ``y`` is neither binary nor multiclass, :class:`KFold` is used.

        Refer :ref:`User Guide <cross_validation>` for the various
        cross-validators that can be used here.

    n_jobs : int or None, optional (default=None)
        Number of jobs to run in parallel.
        ``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.
        ``-1`` means using all processors. See :term:`Glossary <n_jobs>`
        for more details.

    train_sizes : array-like, shape (n_ticks,), dtype float or int
        Relative or absolute numbers of training examples that will be used to
        generate the learning curve. If the dtype is float, it is regarded as a
        fraction of the maximum size of the training set (that is determined
        by the selected validation method), i.e. it has to be within (0, 1].
        Otherwise it is interpreted as absolute sizes of the training sets.
        Note that for classification the number of samples usually have to
        be big enough to contain at least one sample from each class.
        (default: np.linspace(0.1, 1.0, 5))
    """
    if axes is None:
        _, axes = plt.subplots(1, 3, figsize=(20, 5))

    axes[0].set_title(title)
    if ylim is not None:
        axes[0].set_ylim(*ylim)
    axes[0].set_xlabel("Training examples")
    axes[0].set_ylabel("Score")

    train_sizes, train_scores, test_scores, fit_times, _ = \
        learning_curve(estimator, X, y, cv=cv, n_jobs=n_jobs,
                       train_sizes=train_sizes,
                       return_times=True)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    fit_times_mean = np.mean(fit_times, axis=1)
    fit_times_std = np.std(fit_times, axis=1)

    # Plot learning curve
    axes[0].grid()
    axes[0].fill_between(train_sizes, train_scores_mean - train_scores_std,
                         train_scores_mean + train_scores_std, alpha=0.1,
                         color="r")
    axes[0].fill_between(train_sizes, test_scores_mean - test_scores_std,
                         test_scores_mean + test_scores_std, alpha=0.1,
                         color="g")
    axes[0].plot(train_sizes, train_scores_mean, 'o-', color="r",
                 label="Training score")
    axes[0].plot(train_sizes, test_scores_mean, 'o-', color="g",
                 label="Cross-validation score")
    axes[0].legend(loc="best")

    # Plot n_samples vs fit_times
    axes[1].grid()
    axes[1].plot(train_sizes, fit_times_mean, 'o-')
    axes[1].fill_between(train_sizes, fit_times_mean - fit_times_std,
                         fit_times_mean + fit_times_std, alpha=0.1)
    axes[1].set_xlabel("Training examples")
    axes[1].set_ylabel("fit_times")
    axes[1].set_title("Scalability of the model")

    # Plot fit_time vs score
    axes[2].grid()
    axes[2].plot(fit_times_mean, test_scores_mean, 'o-')
    axes[2].fill_between(fit_times_mean, test_scores_mean - test_scores_std,
                         test_scores_mean + test_scores_std, alpha=0.1)
    axes[2].set_xlabel("fit_times")
    axes[2].set_ylabel("Score")
    axes[2].set_title("Performance of the model")

    return plt



from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC as SVM
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier

fig, axes = plt.subplots(3, 5, figsize=(20, 25))

title = "Learning Curves (LogisticRegression)"
# Cross validation with 100 iterations to get smoother mean test and train
# score curves, each time with 20% data randomly selected as a validation set.
cv = ShuffleSplit(n_splits=10, test_size=0.2, random_state=0)

estimator = LogisticRegression()
plot_learning_curve(estimator, title, X, y, axes=axes[:, 0], ylim=(0.0, 1.01),
                    cv=cv, n_jobs=4)


title = "Learning Curves (SVM)"
estimator = SVM(gamma=0.001)
plot_learning_curve(estimator, title, X, y, axes=axes[:, 1], ylim=(0.0, 1.01),
                    cv=cv, n_jobs=4)



title = "Learning Curves (GaussianNB)"
estimator = GaussianNB()
plot_learning_curve(estimator, title, X, y, axes=axes[:, 2], ylim=(0.0, 1.01),
                    cv=cv, n_jobs=4)

title = "Learning Curves (DT)"
estimator = DecisionTreeClassifier(max_depth=3)
plot_learning_curve(estimator, title, X, y, axes=axes[:, 3], ylim=(0.0, 1.01),
                    cv=cv, n_jobs=4)


title = "Learning Curves (MLP)"
estimator = MLPClassifier(solver='lbfgs', alpha=1e-5,
                    hidden_layer_sizes=(5, 2), random_state=1)
plot_learning_curve(estimator, title, X, y, axes=axes[:, 4], ylim=(0.0, 1.01),
                    cv=cv, n_jobs=4)

plt.show()